In [1]:
!pip3 install beautifulsoup4
!pip3 install requests


In [2]:
from bs4 import BeautifulSoup 
import requests 
import re
import unicodedata
import pandas as pd


In [3]:
#Definifn some helper functin 
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out
def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def get_customer(table_cells):
    if table_cells.a:
        Customer = table_cells.a.string
    else:
        Customer = None
    return Customer

def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell.
    It extracts and cleans up extra HTML elements.
    """
    # Remove <br>, <a>, and <sup> if present
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    # Join only the string parts of the contents (no need for extra get_text checks)
    colunm_name = ' '.join([content.strip() for content in row.contents if isinstance(content, str)])

    # Filter out any numeric-only or empty column names
    if not colunm_name.strip().isdigit():
        colunm_name = colunm_name.strip()
        return colunm_name
    return None



In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [10]:
#Sending a get request and Extracting the Tables 
Response = requests.get(static_url)
Response.headers['Content-type']
Tables = Response.text

#Creating the beautifulsoup object
Soup = BeautifulSoup(Tables, 'html.parser')
Soup.title

#Putting the column names from a table in a list 
html_tables = Soup.find_all('table')
row = html_tables[2].find('tr')
header = row.find_all('th')
# Extracting the cleaned-up column names
column_names = [extract_column_from_header(cell) for cell in header if extract_column_from_header(cell)]

# Print the extracted column names
print(column_names)


#Creating a dictionary from the columns extracted to be used as a dataframe
launch_dict= dict.fromkeys(column_names)
launch_dict

#Deleting irrelevant columns
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

launch_dict

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


{'Flight No.': [],
 'Launch site': [],
 'Payload': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launch outcome': [],
 'Version Booster': [],
 'Booster landing': [],
 'Date': [],
 'Time': []}

In [33]:
#Extracting the data from the table
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(Soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            #Appending necessary data to a designated dictionary 
            launch_dict['Flight No.'].append(flight_number)            #print(flight_number)
            datatimelist=date_time(row[0])
            
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            customer = get_customer(row[6])
            launch_dict['Customer'].append(customer)
           
            
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
          
            
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

In [37]:
#Creating a dataframe from the dictionary
spacex_df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })


In [40]:
spacex_df.describe()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
count,1105,681,575,575,469,461,242,893,121,1105,999
unique,121,5,82,84,8,37,3,55,8,121,116
top,1,CCAFS,Starlink,"15,600 kg",LEO,NASA,Success\n,F9 B5,Success,4 June 2010,04:45
freq,10,240,121,98,254,126,176,147,80,10,27
